# 06 Recommendation Systems
__Math 3280 - Data Mining__ : Snow College : Dr. Michael E. Olson

* Leskovec, Chapter 9
-----

In [2]:
import numpy as np
import pandas as pd

In [109]:
## Utility Matrix

movies = ['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','M13','M14','M15']

ratings = {
    'User A' : [0,1,1,1,0,1,1,0,1,0,0,1,1,1,1],
    'User B' : [1,1,0,0,0,0,1,1,0,1,0,1,0,0,1],
    'User C' : [0,0,0,1,1,0,0,0,0,1,0,0,0,0,1]
}

user_ratings = pd.DataFrame(ratings).transpose()
user_ratings.columns = movies

user_ratings

,M01,M02,M03,M04,M05,M06,M07,M08,M09,M10,M11,M12,M13,M14,M15
User A,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1
User B,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1
User C,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1


In [112]:
## Item Profile : Movie Database
actors_list = ['Julia Roberts','Robin Williams','Clint Eastwood','Ian McKellen','Movie Rating']

actors = {
    'M01' : [1,0,0,1,3],
    'M02' : [0,0,1,0,5],
    'M03' : [1,0,0,0,4],
    'M04' : [0,1,0,0,2],
    'M05' : [0,1,0,0,4],
    'M06' : [0,0,1,0,5],
    'M07' : [1,1,0,0,3],
    'M08' : [1,0,0,1,1],
    'M09' : [0,0,1,1,5],
    'M10' : [1,1,0,0,5],
    'M11' : [1,0,0,0,1],
    'M12' : [0,0,0,1,2],
    'M13' : [0,1,0,1,2],
    'M14' : [0,1,1,0,5],
    'M15' : [0,0,1,1,4]
}

movie_casts = pd.DataFrame(actors).transpose()
movie_casts.columns = actors_list

movie_casts

,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen,Movie Rating
M01,1,0,0,1,3
M02,0,0,1,0,5
M03,1,0,0,0,4
M04,0,1,0,0,2
M05,0,1,0,0,4
M06,0,0,1,0,5
M07,1,1,0,0,3
M08,1,0,0,1,1
M09,0,0,1,1,5
M10,1,1,0,0,5


In [127]:
def cosine_distance(item_1,item_2):
    dotproduct = np.dot(movie_casts.loc[item_1], movie_casts.loc[item_2])
    norms = np.dot(movie_casts.loc[item_1], movie_casts.loc[item_1])*np.dot(movie_casts.loc[item_2], movie_casts.loc[item_2])
    return dotproduct/np.sqrt(norms)

In [89]:
## Example 9.2
alpha = 2
item_1 = 'M07'
item_2 = 'M10'

movie_comparison = movie_casts[[item_1, item_2]]
movie_comparison['Movie Rating'] = movie_comparison['Movie Rating']*alpha
movie_comparison

,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen,Movie Rating
M07,1,1,0,0,6
M10,1,1,0,0,10


### User Profiles

In [131]:
## User Profile
## Example 9.3 : Movies given only likes or dislikes
dotproduct = np.dot(movie_casts['Julia Roberts'], user_ratings.loc['User A'])
norm = np.dot(user_ratings.loc['User A'], user_ratings.loc['User A'])
print(f"Dot Product of User Likes and Julia Roberts' Movies : {dotproduct}")
print(f"Norm of User Likes (number of movies liked)         : {norm}")
print(f"User weight to movies with Julia Roberts            : {dotproduct / norm}\n")


user_profile_likes = pd.DataFrame(columns=actors_list)

for actor_id in actors_list:
    for user_id in user_ratings.index:
        user_profile_likes.loc[user_id,actor_id] = np.dot(movie_casts[actor_id], user_ratings.loc[user_id]) / user_ratings.loc[user_id].sum()
        
user_profile_likes

Dot Product of User Likes and Julia Roberts' Movies : 2
Norm of User Likes (number of movies liked)         : 10
User weight to movies with Julia Roberts            : 0.2



,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen,Movie Rating
User A,0.2,0.4,0.5,0.4,3.7
User B,0.571429,0.285714,0.285714,0.571429,3.285714
User C,0.25,0.75,0.25,0.25,3.75


In [146]:
## User Profile (Average rating)

### Start by finding the movies where the user has given a rating and the actor is in the movie
movie_casts['Julia Roberts'] * user_ratings.loc['User A']

M01    0
M02    0
M03    1
M04    0
M05    0
M06    0
M07    1
M08    0
M09    0
M10    0
M11    0
M12    0
M13    0
M14    0
M15    0
dtype: int64

In [147]:
### We can get the count by taking the dot product
cnt = np.dot(movie_casts['Julia Roberts'], user_ratings.loc['User A'])
cnt

2

In [143]:
### Now, find the average rating the user gave those movies
movie_casts['Julia Roberts'] * user_ratings.loc['User A'] * movie_casts['Movie Rating']
movie_casts[condition.values]

KeyError: "None of [Int64Index([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype='int64')] are in the [columns]"

In [ ]:
movie_casts['Julia Roberts'] * user_ratings.loc['User A'] * movie_casts['Movie Rating']

In [ ]:


user_profile_ratings = pd.DataFrame(columns=actors_list)

for actor_id in actors_list:
    for user_id in user_ratings.index:
        user_profile_ratings.loc[user_id,actor_id] = 
        
user_profile_ratings

In [133]:
user_ratings

,M01,M02,M03,M04,M05,M06,M07,M08,M09,M10,M11,M12,M13,M14,M15
User A,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1
User B,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1
User C,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1


In [128]:
distances = pd.DataFrame(columns=movies)
for item1 in movies:
    for item2 in movies:
        distances.loc[item1,item2] = cosine_distance(item1,item2)

distances

,M01,M02,M03,M04,M05,M06,M07,M08,M09,M10,M11,M12,M13,M14,M15
M01,1.0,0.886969,0.950654,0.80904,0.877527,0.886969,0.909091,0.870388,0.928414,0.928414,0.852803,0.94388,0.86164,0.870388,0.92387
M02,0.886969,1.0,0.951303,0.877058,0.951303,1.0,0.886969,0.566139,0.981307,0.943564,0.693375,0.877058,0.800641,0.981307,0.970725
M03,0.950654,0.951303,1.0,0.867722,0.941176,0.951303,0.950654,0.70014,0.93352,0.980196,0.857493,0.867722,0.792118,0.93352,0.914659
M04,0.80904,0.877058,0.867722,1.0,0.976187,0.877058,0.94388,0.516398,0.860663,0.946729,0.632456,0.8,0.912871,0.946729,0.843274
M05,0.877527,0.951303,0.941176,0.976187,1.0,0.951303,0.950654,0.560112,0.93352,0.980196,0.685994,0.867722,0.891133,0.980196,0.914659
M06,0.886969,1.0,0.951303,0.877058,0.951303,1.0,0.886969,0.566139,0.981307,0.943564,0.693375,0.877058,0.800641,0.981307,0.970725
M07,0.909091,0.886969,0.950654,0.94388,0.950654,0.886969,1.0,0.696311,0.870388,0.98644,0.852803,0.80904,0.86164,0.928414,0.852803
M08,0.870388,0.566139,0.70014,0.516398,0.560112,0.566139,0.696311,1.0,0.666667,0.666667,0.816497,0.774597,0.707107,0.555556,0.680414
M09,0.928414,0.981307,0.93352,0.860663,0.93352,0.981307,0.870388,0.666667,1.0,0.925926,0.680414,0.946729,0.864242,0.962963,0.99794
M10,0.928414,0.943564,0.980196,0.946729,0.980196,0.943564,0.98644,0.666667,0.925926,1.0,0.816497,0.860663,0.864242,0.962963,0.907218


In [45]:
## Item (Movie) Profile
item_profile = movie_casts.loc['Julia Roberts']
item_profile

M01    1
M02    0
M03    1
M04    0
M05    0
M06    0
M07    1
M08    1
M09    0
M10    1
M11    1
M12    0
M13    0
M14    0
M15    0
Name: Julia Roberts, dtype: int64

In [46]:
## User A Ratings
userA_profile = user_ratings.loc['User A']
userA_profile

M01    0
M02    1
M03    1
M04    1
M05    0
M06    1
M07    1
M08    0
M09    1
M10    0
M11    0
M12    1
M13    1
M14    1
M15    1
Name: User A, dtype: int64

In [58]:
## User Rating
## Rating of 1-5 stars : 0 means no rating
userA_ratings = pd.DataFrame(np.random.randint(0,5,15), index=movies, columns=['User A'])
userA_ratings

,User A
M01,1
M02,3
M03,2
M04,2
M05,4
M06,3
M07,0
M08,1
M09,2
M10,1


In [59]:
## Show only the ratings for movies with Julia Roberts
condition = userA_ratings['User A'] * item_profile > 0
userA_ratings[condition]

,User A
M01,1
M03,2
M08,1
M10,1
M11,3


In [60]:
## Ratings - average rating
userA_ratings[condition] - userA_ratings.mean()

,User A
M01,-0.866667
M03,0.133333
M08,-0.866667
M10,-0.866667
M11,1.133333
